# digest_files( )
Função para automação da ingestão de arquivos para o notebook, criação de dfs e armazenamento de arquivos e dfs nos dicionários corretos.

In [ ]:
# Cria função para digestão de arquivos e armazenamento de dfs nos dicionários do projeto

''' DOC:
Lê e armazena arquivos de dados com pd.read_ listados em uma lista passada como argumento.
Distingue entre formatos de arquivo .csv ou .xlsx/.xls. 
Arquivos .xls são lidos e armazenados sem intervenções. 
Em arquivos .csv é identificado o tipo de encoding utilizado com chardet.detect() e o arquivo é lido e tem os separadores substituídos por ',' antes do armazenamento.
Gera nomes dinamicamente, padronizando texto removendo espaços e hífens.
Armazena df em variável no escopo global com globals().
Adiciona arquivos nos dicionários do projeto.

Argumento obrigatório:
Files: Lista com nomes de arquivos para leitura.
'''

def digest_files(Files): 

    # Gerando listas vazia para armazenar com os dfs criados, nomes dos dfs criados e nomes dos arquivos importados
    Lista_dfs            = []
    Lista_nomes_dfs      = []
    Lista_imported_files = []
    
    for file in  Files: 

        # Detecta formato de arquivo.csv          
        if '.csv' in file: 

            # Produz nomes dinamicamente para o df a partir do arquivo 
            name = file.replace('.csv', '').replace(' ', '_').replace('-', '_')
            df   = f'raw_{name}'

            # Tenta criar o df e armazenar em var globalmente a partir de .csv sem intervenções
            try: 
                globals()[df] = pd.read_csv(file)
            
            # Em caso de erro tenta especificar encoding e substituir separadores específicos por ','
            except: 
                try: 
                    # Verifica o tipo de encoding do arquivo .csv e armazena em variável para uso na função pd.read_csv
                    with open (file, 'rb') as f: 
                        encod_result = chardet.detect(f.read())['encoding']

                    # abre o arquivo e substitui os separadores em cada linha antes de pd.read_csv, adiciona o tipo de enconding na função
                    globals()[df] = pd.read_csv(StringIO(''.join(l.replace('\t', ',').replace(';', ',') for l in open(file))), encoding = encod_result)

                    # Adiciona df criado nas listas da função
                    Lista_dfs.append(globals()[df])
                    Lista_nomes_dfs.append(df)
                    Lista_imported_files.append(file)
                    print(f'{df} criado com sucesso a partir do arquivo {file}, encoding detectado: {encod_result}')                      

                # Em caso de persistência de erro, não interrompe o processo e armazena arquivo não processado na lista de erros
                except: 
                    print(f'Impossível criar df a partir de {file}')
                    Errors_list.append(file)

            else: 
                # Adiciona df criado nas listas da função
                Lista_dfs.append(globals()[df])
                Lista_nomes_dfs.append(df)
                Lista_imported_files.append(file)
                print(f'{df} criado com sucesso a partir do arquivo {file}')        

        # Detecta formato de arquivo.xls ou .xlsx
        elif '.xlsx' or '.xls' in file: 
            
            # Produz nomes dinamicamente e cria variável que armazena o conteúdo do arquivo em df no escopo global
            name = file.replace('.xlsx', '').replace('.xls', '').replace(' ', '_').replace('-','_')
            df   = f'raw_{name}'

            try: 
                globals()[df] = pd.read_excel(file)
            except: 
                print(f'Impossível criar df a partir de {file}')
                Errors_list.append(file)
            else: 
                # Adiciona df criado nas listas da função
                Lista_dfs.append(globals()[df])
                Lista_nomes_dfs.append(df)
                Lista_imported_files.append(file)
                print(f'{df} criado com sucesso a partir do arquivo {file}')       

        else: 
            print(f'{file} não é um arquivo .csv, .xls ou .xlsx')

    # Zipa listas da função em dicionários utilizados para atualizar os dicionários do projeto
    dict_update_line = dict(zip(Lista_nomes_dfs, Lista_imported_files))
    Original_Files.update(dict_update_line)
    dict_update_line = dict(zip(Lista_nomes_dfs, Lista_dfs))
    Raw_DFs.update(dict_update_line)
    

# NaN_rows( )
Função para visualização rápida de rows específica que contém Nulls em coluna especificada.

In [ ]:
# Cria função para visualização rápida de rows específicas que contém Nulls em colunas específicas
''' DOC:
NaN_rows(df, col) recebe como argumentos um dataframe (var) e uma coluna específica (str).
Cria uma máscara que retorna apenas as linhas com NaN na coluna indicada, sem eliminar outras colunas.
'''

def NaN_rows(df, col):
    mask = df[col].isna()
    NaN_rows = df[mask]
    return NaN_rows


# add_IBGE_std( )

Função para adicionar nomenclatura padronizada aos dfs a partir do padrão IBGE mapeado nos dicts CodMunic_dict e CodUF_dict

In [ ]:
# Cria função para adicionar nomenclatura aos dfs a partir do padrão IBGE mapeado nos dicts CodMunic_dict e CodUF_dict
''' DOC:
add_IBGE_std(df, col, cod_ibge = True, drop = []):

Adiciona as colunas cod_municipio, nome_municipio, cod_UF e nome_UF
a partir de coluna do df com código do município ou nome do município 
indicada como parâmetro.

A função necessita dos dicionários criados a partir de tabela do IBGE com 
códigos e nomes dos municípios (CodMunicipio_dict e MunicipioCod_dict) e 
realiza o mapeamento dos valores da coluna indicada como chave.

No caso de scale = 'mun', adiciona nomes e códigos de municípios e UFs à tabela.

No caso de scale = 'UF', adiciona apenas nomes e códigos de UFs à tabela.

No caso de cod_ibge = True, toma o valor dos códigos do município, cria coluna a 
partir do dicionário CodMunicpio_dict, e cria cod_UF e nome_UF a partir do dict 
CodUF_dict

No caso de cod_ibge = False, toma o valor dos nomes dos municípios, cria coluna 
cod_municipio a partir do dicionário MunicipioCod, e cria cod_UF e nome_UF a partir 
do dict CodUF_dict 

O argumento drop indica colunas que se deseja deletar do df durante o processo.

A função demanda como argumentos obrigatórios: 
  - (var) df: Um data frame a ser processado
  - (str) col: O nome de uma coluna chave cujos valores serão mapeados para produção de 
     nova coluna cod_municipio ou cod_IBGE

A função recebe como argumentos opcionais:
  - (str) scale: 'mun' por padrão. 'UF' aplica a padronização e inserção de códigos ou nomes 
     apenas de UFs. 'mun' aplica a padronização e inserção de nomes e códigos de municípios e UFs.
  - (boolean) cod_ibge: True por padrão. True indica que a coluna chave indicada contém os códigos 
     de município do IBGE a partir do qual serão gerados os nomes padronizados. False indica que a 
     coluna contém nomes dos município, a partir dos quais serão gerados os códigos de município.
  - (list of str) drop: [] por padrão. As strings indicadas dentro de drop são deletadas ao final do processo.

'''


def add_IBGE_std(df, col, scale = 'mun', cod_ibge = True, drop = []):
    
    x = df

    if scale == 'mun' and cod_ibge == True :

        # Adiciona colunas numeradas como strings a partir dos códigos do IBGE mapeados como keys em CodMunicipio_dict
        x ['!02cod_municipio']  = x[col].astype(str)
        x ['!03nome_municipio'] = x['!02cod_municipio'].map(CodMunicipio_dict)
        x ['!00cod_UF']         = x[col].astype(str).str[:2]
        x ['!01nome_UF']        = x['!00cod_UF'].map(CodUF_dict)

        # Cria uma lista com os nomes das colunas e a reordena lexicograficamente
        new_order = sorted(x.columns.tolist())

        # Reinsere as colunas no df a partir da lista ordenada new_order
        x = x[new_order]

        # Remove a numeração no nome das colunas
        x.rename({
        '!00cod_UF'        : 'cod_UF',
        '!01nome_UF'       : 'nome_UF',
        '!02cod_municipio' : 'cod_municipio',
        '!03nome_municipio': 'nome_municipio'
        }, axis = 1, inplace = True)

    # Adiciona a colunas a partir do nome do município disponível, padronizado com slugify para corresponder aos nomes padronizados como keys em MunicipioCod_dict   
    elif scale == 'mun' and cod_ibge == False  :
        # Preenche NaN com 'None' para proceder com slugify dos nomes de municipios
        x['!03nome_municipio'] = x[col].fillna('None').apply(slugify)
        x['!02cod_municipio']  = x['!03nome_municipio'].map(MunicipioCod_dict)
        x['!00cod_UF']         = x['!02cod_municipio'].astype(str).str[:2]
        x['!01nome_UF']        = x['!00cod_UF'].map(CodUF_dict)

        # Reordena e renomeia as novas colunas
        new_order = sorted(x.columns.tolist())
        x         = x[new_order]
        x.rename({
        '!00cod_UF'        : 'cod_UF',
        '!01nome_UF'       : 'nome_UF',
        '!02cod_municipio' : 'cod_municipio',
        '!03nome_municipio': 'nome_municipio'
        }, axis = 1, inplace = True)

    elif scale == 'UF' and cod_ibge == True:
        
        # Criando as colunas nome_UF a partir dos códigos do IBGE mapeados como keys em CodUF_dict
        x ['!00cod_UF']   = x[col].astype(str)
        x ['!01nome_UF']  = x['!00cod_UF'].map(CodUF_dict)

        # Reordena e renomeia as novas colunas
        new_order = sorted(x.columns.tolist())
        x         = x[new_order]
        x.rename({
        '!00cod_UF' : 'cod_UF',
        '!01nome_UF': 'nome_UF'
        }, axis = 1, inplace = True)

    # Adiciona colunas a partir do nome do estado disponível, padronizado com slugify correspondendo aos nomes padronizados como keys em UFCod_dict
    elif scale == 'UF' and cod_ibge == False: 
        x['!01nome_UF'] = x[col].fillna('None').apply(slugify)
        x['!00cod_UF']  = x['!01nome_UF'].map(UFCod_dict)

        # Reordena e renomeia as novas colunas
        new_order = sorted(x.columns.tolist())
        x         = x[new_order]
        x.rename({
        '!00cod_UF' : 'cod_UF',
        '!01nome_UF': 'nome_UF'
        }, axis = 1, inplace = True)

    elif scale == 'all' and cod_ibge == True:

        # Adiciona colunas numeradas como strings nome_municipio e nome_UF a partir dos códigos do IBGE mapeados como keys em CodMunicipio_dict
        x['!04cod_municipio']      = x[col].astype(str)
        x['!05nome_municipio']     = x['!04cod_municipio'].map(CodMunicipio_dict)
        x['!00cod_UF']             = x[col].astype(str).str[:2]
        x['!01nome_UF']            = x['!00cod_UF'].map(CodUF_dict)
        x['!02nome_mesorregiao']   = x['!04cod_municipio'].map(CodMeso_dict)
        x['!03nome_microrregiao']  = x['!04cod_municipio'].map(CodMicro_dict)
        
        # Cria uma lista com os nomes das colunas e a reordena lexicograficamente
        new_order = sorted(x.columns.tolist())

        # Reinsere as colunas no df a partir da lista ordenada new_order
        x = x[new_order]

        # Remove a numeração no nome das colunas
        x.rename({
        '!00cod_UF'           : 'cod_UF',
        '!01nome_UF'          : 'nome_UF',
        '!04cod_municipio'    : 'cod_municipio',
        '!05nome_municipio'   : 'nome_municipio',
        '!02nome_mesorregiao' : 'nome_mesorregiao',
        '!03nome_microrregiao': 'nome_microrregiao'
        }, axis = 1, inplace = True)

        # Adiciona a colunas a partir do nome do município disponível, padronizado com slugify para corresponder aos nomes padronizados como keys em MunicipioCod_dict
    elif scale == 'all' and cod_ibge == False: 
        x['!05nome_municipio']    = x[col].fillna('None').apply(slugify)
        x['!04cod_municipio']     = x['!05nome_municipio'].map(MunicipioCod_dict)
        x['!00cod_UF']            = x['!04cod_municipio'].astype(str).str[:2]
        x['!01nome_UF']           = x['!00cod_UF'].map(CodUF_dict)
        x['!02nome_mesorregião']  = x['!04cod_municipio'].map(CodMeso_dict)
        x['!03nome_microrregião'] = x['!04cod_municipio'].map(CodMicro_dict)

        # Reordena e renomeia as novas colunas
        new_order = sorted(x.columns.tolist())
        x         = x[new_order]
        x.rename({
        '!00cod_UF'           : 'cod_UF',
        '!01nome_UF'          : 'nome_UF',
        '!04cod_municipio'    : 'cod_municipio',
        '!05nome_municipio'   : 'nome_municipio',
        '!02nome_mesorregião' : 'nome_mesorregiao',
        '!03nome_microrregião': 'nome_microrregiao'
        }, axis = 1, inplace = True)

    # Se o parâmetro drop for declarado como argumento, deleta as colunas listadas do df
    if len(drop) != 0 :
      x.drop(drop, axis = 1, inplace = True)

    return x


# make_filtered_df( )
Função para aplicar múltiplos filtros semelhantes em um DF e armazenar DFs filtrados em arquivos.xlsx


In [ ]:
# Cria função para aplicar múltiplos filtros semelhantes em um DF e armazenar DFs filtrados em arquivos.xlsx
''' Function DOC:
make_filtered_df(col, filter_value, df_name, [Project_DFs_number, save_excel, save_path])

Gera os nomes e DFs filtrados dinamicamente, armazena o resultado no dict Filtered_DFs e opcionalmente salva os DFs em arquivos .xlsx

A função demanda como argumentos obrigatórios: 
  - str col: o nome de uma coluna
  - str filter_value: uma lista com valores de filtros
  - str df_name: o nome do DF conforme armazenado como key em um dict em Project_DFs
A função recebe como argumentos opcionais:
  - int Project_DFs_number: o índice de um dict em Project_DFs, por padrão definido em 3. [2] Raw_DFs; [3] Working_DFs; [4] Treated_DFs; [5] Filtered_DFs. [0] e [1] resultarão em erro.
  - Boolean save_excel:True salva o df em um arquivo .xlsx no cwd ou diretório indicado. Por padrão definido como False, não salva arquivo .xlsx
  - str save_path: indica diretório para salvar arquivo .xlsx com o novo DF. Por padrão determina o cwd com a função os.getcwd()

'''

def make_filtered_dfs(df_name, col, filter_values, dict_number = 3, save_excel=False, save_path = os.getcwd()) :

    # Cria novo nome para o novo df unindo o df original ao filtro indicado
    new_df_name = df_name + '_' + '_'.join(filter_values)
    new_df_name = new_df_name.replace(' ','_').replace('-','_')

    # Seleciona o df indicado em df_name no dicionário indicado
    df = Project_DFs[dict_number][df_name]

    # Cria novo df globalmente a partir de lista de filtros passada como argumento filter_values
    globals()[new_df_name] = df[df[col].isin(filter_values)]
    Filtered_DFs[new_df_name] = globals()[new_df_name]


    if save_excel == True :
      globals()[new_df_name].to_excel(f'{save_path}\\{new_df_name}.xlsx', index = False)

      return print(f'''
                    {new_df_name} gerado com sucesso.
                    Shape: {globals()[new_df_name].shape}
                    Filtered_DFs atualizado com sucesso.
                    {print(Filtered_DFs.keys())}
                    Arquivo {new_df_name}.xlsx salvo com sucesso em {save_path}
                  ''')
    else :
       return print(f'''
                    {new_df_name} gerado com sucesso.
                    Shape: {globals()[new_df_name].shape}
                    Filtered_DFs atualizado com sucesso.
                    {print(Filtered_DFs.keys())}
                  ''')
       

# save_graph( )
Função para salvar gráficos com a mesma configuração básica no mesmo save_path

In [ ]:
def save_graph(title, scope):
    scope = re.sub(r'\n', ' ', scope) # re module necessário para lidar com \n em nomes de municípios transformados
    plt.savefig(f'{save_path}\\{title}_{scope}.png', dpi = 150, bbox_inches='tight' )